Exploring Indian cities for presence of Indian restaurants

In [124]:

import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests 

!pip install bs4

from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print("Libraries imported.")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


Import CSV file consisting of Indian cities and longitude, latitude

In [125]:
filename = "/resources/data/in.csv"
in_df = pd.read_csv(filename)
in_df.head(in_df.shape[0])

,city,lat,lng
0,Mumbai,18.987807,72.836447
1,Delhi,28.651952,77.231495
2,Kolkata,22.562627,88.363044
3,Chennai,13.084622,80.248357
4,Bengaluru,12.977063,77.587106
5,Hyderabad,17.384052,78.456355
6,Ahmedabad,23.025793,72.587265
7,Haora,22.576882,88.318566
8,Pune,18.513271,73.849852
9,Lucknow,26.839281,80.923133


Initiate Four square and Search for venues in a 15 km radius of each city

In [126]:
# define Foursquare Credentials and Version
CLIENT_ID = '5QOUVRBWZQWZVWLO4WTQT5OPHHE4F5WHPKAXRH1I2ARB1OLL' # your Foursquare ID
CLIENT_SECRET = '1BRWFFULZEEPFDUBTCGGL4HPBDLN1MEIUBD1BXXOYWF0CEVQ' # your Foursquare Secret
VERSION = '20200223' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5QOUVRBWZQWZVWLO4WTQT5OPHHE4F5WHPKAXRH1I2ARB1OLL
CLIENT_SECRET:1BRWFFULZEEPFDUBTCGGL4HPBDLN1MEIUBD1BXXOYWF0CEVQ


In [127]:
radius = 15000
LIMIT = 200

venues = []

for lat, long, city in zip(in_df['lat'], in_df['lng'], in_df['city']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            city,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [135]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['City', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2638, 7)


,City,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Mumbai,18.987807,72.836447,The St. Regis Mumbai,18.993652,72.825220,Hotel
1,Mumbai,18.987807,72.836447,High Street Phoenix,18.994967,72.825032,Shopping Mall
2,Mumbai,18.987807,72.836447,Jai Hind Lunch Home,19.002183,72.829512,Seafood Restaurant
3,Mumbai,18.987807,72.836447,Smoke House Deli,18.994478,72.824400,Restaurant
4,Mumbai,18.987807,72.836447,The Sahib Room & Kipling Bar,18.994333,72.823850,Indian Restaurant


Group cities

In [136]:
venues_df.groupby(["City"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
City,,,,,,
Ahmadnagar,6,6,6,6,6,6
Ahmedabad,100,100,100,100,100,100
Aizawl,5,5,5,5,5,5
Akola,5,5,5,5,5,5
Alwar,7,7,7,7,7,7
Bareilly,4,4,4,4,4,4
Belgaum,21,21,21,21,21,21
Bellary,4,4,4,4,4,4
Bengaluru,100,100,100,100,100,100


In [137]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 217 uniques categories.


In [138]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:144]

array(['Hotel', 'Shopping Mall', 'Seafood Restaurant', 'Restaurant',
       'Indian Restaurant', 'Deli / Bodega', 'Lounge', 'Club House',
       'Nightclub', 'Scenic Lookout', 'Golf Course', 'Multiplex', 'Bar',
       'Asian Restaurant', 'Fast Food Restaurant', 'Gourmet Shop',
       'Music Venue', 'Ice Cream Shop', 'Bakery',
       'Vegetarian / Vegan Restaurant', 'Café',
       'Middle Eastern Restaurant', 'Playground', 'Italian Restaurant',
       'Brewery', 'Juice Bar', 'Coffee Shop', 'French Restaurant',
       "Men's Store", 'Sandwich Place', 'Cricket Ground',
       'History Museum', 'Chinese Restaurant', "Women's Store", 'Beach',
       'Park', 'Snack Place', 'Performing Arts Venue', 'Breakfast Spot',
       'Art Gallery', 'Parsi Restaurant', 'Diner', 'Mexican Restaurant',
       'Boutique', 'Theater', 'Pub', 'Pizza Place', 'Tea Room',
       'New American Restaurant', 'Plaza', 'Monument / Landmark',
       'Food & Drink Shop', 'Food Truck', 'South Indian Restaurant',
       'M

Encoding

In [139]:
# one hot encoding
in_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add city column back to dataframe
in_onehot['City'] = venues_df['City'] 

# move city column to the first column
fixed_columns = [in_onehot.columns[-1]] + list(in_onehot.columns[:-1])
in_onehot = in_onehot[fixed_columns]

print(in_onehot.shape)
in_onehot.head()

(2638, 217)


,Zoo,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Awadhi Restaurant,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Chaat Place,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,English Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Heliport,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Lake,Lighthouse,Lounge,Maharashtrian Restaurant,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Northeast Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Palace,Park,Parsi Restaurant,Performing Arts Venue,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Town,Track Stadium,Trail,Train Station,Travel & Transport,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Grouping of venues

In [150]:
in_grouped = in_onehot.groupby(["City"]).mean().reset_index()

print(in_grouped.shape)
in_grouped

(74, 217)


,City,Zoo,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Awadhi Restaurant,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,English Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Heliport,Hindu Temple,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Lake,Lighthouse,Lounge,Maharashtrian Restaurant,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Northeast Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Palace,Park,Parsi Restaurant,Performing Arts Venue,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Town,Track Stadium,Trail,Train Station,Travel & Transport,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Women's Store
0,Ahmadnagar,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00

New data frame

In [165]:
in_restaurant = in_grouped[["City","Indian Restaurant"]]
in_restaurant.head()



K Means clustering

In [153]:
# set number of clusters
kclusters = 4

in_clustering = in_restaurant.drop(["City"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(in_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 0, 0, 0, 0, 2, 3, 2, 0], dtype=int32)

In [154]:
# create a new dataframe that includes the cluster 
in_merged = in_restaurant.copy()

# add clustering labels
in_merged["Cluster Labels"] = kmeans.labels_

In [155]:
in_merged.rename(columns={"City": "City"}, inplace=True)
in_merged.head()

,City,Indian Restaurant,Cluster Labels
0,Ahmadnagar,0.333333,3
1,Ahmedabad,0.130000,1
2,Aizawl,0.000000,0
3,Akola,0.000000,0
4,Alwar,0.000000,0


In [156]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
in_merged = in_merged.join(in_df.set_index("city"), on="City")

print(in_merged.shape)
in_merged.head() # check the last columns!

(74, 5)


,City,Indian Restaurant,Cluster Labels,lat,lng
0,Ahmadnagar,0.333333,3,19.094571,74.738432
1,Ahmedabad,0.130000,1,23.025793,72.587265
2,Aizawl,0.000000,0,23.736701,92.714596
3,Akola,0.000000,0,20.709569,76.998103
4,Alwar,0.000000,0,27.566291,76.610202


In [157]:
# sort the results by Cluster Labels
print(in_merged.shape)
in_merged.sort_values(["Cluster Labels"], inplace=True)
in_merged

(74, 5)


,City,Indian Restaurant,Cluster Labels,lat,lng
36,Khammam,0.000000,0,17.247672,80.143682
31,Imphal,0.000000,0,24.808053,93.944203
40,Kota,0.000000,0,25.182544,75.839065
25,Gwalior,0.000000,0,26.229825,78.173369
24,Gulbarga,0.000000,0,17.335827,76.837570
23,Gaya,0.000000,0,24.796858,85.003852
49,Muzaffarpur,0.000000,0,26.122593,85.390553
20,Cuttack,0.000000,0,20.522922,85.788130
53,Parbhani,0.000000,0,19.268553,76.770807
57,Raipur,0.031250,0,21.233333,81.633333


In [160]:
# create map
map_clusters = folium.Map(location=[20.5937, 78.9629], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(in_merged['lat'], in_merged['lng'], in_merged['City'], in_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

,City,Indian Restaurant,Cluster Labels,lat,lng
36,Khammam,0.00000,0,17.247672,80.143682
31,Imphal,0.00000,0,24.808053,93.944203
40,Kota,0.00000,0,25.182544,75.839065
25,Gwalior,0.00000,0,26.229825,78.173369
24,Gulbarga,0.00000,0,17.335827,76.837570
23,Gaya,0.00000,0,24.796858,85.003852
49,Muzaffarpur,0.00000,0,26.122593,85.390553
20,Cuttack,0.00000,0,20.522922,85.788130
53,Parbhani,0.00000,0,19.268553,76.770807
57,Raipur,0.03125,0,21.233333,81.633333


,City,Indian Restaurant,Cluster Labels,lat,lng
72,Vadodara,0.142857,1,22.299405,73.208119
42,Lucknow,0.169492,1,26.839281,80.923133
45,Mathura,0.181818,1,27.503501,77.672145
46,Meerut,0.200000,1,28.980018,77.706356
64,Shimoga,0.142857,1,13.932424,75.572555
68,Tirupati,0.178571,1,13.635505,79.419888
52,New Delhi,0.150000,1,28.600000,77.200000
66,Thiruvananthapuram,0.154930,1,8.485498,76.949238
54,Porbandar,0.125000,1,21.641346,69.600868
34,Jammu,0.166667,1,32.735686,74.869112


,City,Indian Restaurant,Cluster Labels,lat,lng
11,Bhiwandi,0.117647,2,19.300229,73.058813
12,Bhopal,0.078431,2,23.254688,77.402892
6,Belgaum,0.047619,2,15.862643,74.508534
13,Bhubaneshwar,0.050847,2,20.272411,85.833853
70,Udaipur,0.068966,2,24.579510,73.690508
16,Chandigarh,0.076923,2,30.736292,76.788398
55,Puducherry,0.096774,2,11.933812,79.829792
32,Indore,0.089552,2,22.717736,75.858590
8,Bengaluru,0.100000,2,12.977063,77.587106
62,Shiliguri,0.058824,2,26.710035,88.428512
